# Data Loading, Storage, and File Formats


In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Reading and Writing Data in Text Format

![T6-1](images/T6-1.png)
![T6-1-2](images/T6-1-2.png)

In [23]:
list(open('examples/ex1.csv'))

['a,b,c,d,message\n', '1,2,3,4,hello\n', '5,6,7,8,world\n', '9,10,11,12,foo']

In [8]:
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [21]:
list(open('examples/ex2.csv'))

['1,2,3,4,hello\n', '5,6,7,8,world\n', '9,10,11,12,foo']

In [10]:
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [12]:
pd.read_csv('examples/ex2.csv', names = ['a','b','c','d', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [13]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [20]:
list(open('examples/csv_mindex.csv'))

['key1,key2,value1,value2\n',
 'one,a,1,2\n',
 'one,b,3,4\n',
 'one,c,5,6\n',
 'one,d,7,8\n',
 'two,a,9,10\n',
 'two,b,11,12\n',
 'two,c,13,14\n',
 'two,d,15,16\n']

In [15]:
pd.read_csv('examples/csv_mindex.csv', index_col = ['key1', 'key2'])

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In some cases, a table might not have a fixed delimiter, using whitespace or some other pattern to separate fields. Consider a text file that looks like this:

(Page 170). 

In [18]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

While you could do some munging by hand, the fields here are separated by a vari‐ able amount of whitespace. In these cases, you can pass a regular expression as a delimiter for read_table. This can be expressed by the regular expression \s+, so we have then:

(Page 171). 

In [29]:
pd.read_csv('examples/ex3.txt', sep='\s+')

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


> Because there was one fewer column name than the number of data rows, read_csv infers that the first column should be the DataFrame’s index in this spe‐ cial case.



The parser functions have many additional arguments to help you handle the wide variety of exception file formats that occur (see a partial listing in Table 6-2). For example, you can skip the first, third, and fourth rows of a file with skiprows:

(Page 171). 

In [26]:
list(open('examples/ex4.csv'))

['# hey!\n',
 'a,b,c,d,message\n',
 '# just wanted to make things more difficult for you\n',
 '# who reads CSV files with computers, anyway?\n',
 '1,2,3,4,hello\n',
 '5,6,7,8,world\n',
 '9,10,11,12,foo']

In [30]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Handling missing values is an important and frequently nuanced part of the file pars‐ ing process. Missing data is usually either not present (empty string) or marked by some sentinel value. By default, pandas uses a set of commonly occurring sentinels, such as NA and NULL:

(Page 171). 

In [32]:
list(open('examples/ex5.csv'))

['something,a,b,c,d,message\n',
 'one,1,2,3,4,NA\n',
 'two,5,6,,8,world\n',
 'three,9,10,11,12,foo']

In [33]:
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [40]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [39]:
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [41]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


![T6-2](images/T6-2.png)
![T6-2-1](images/T6-2-1.png)

### Reading Text Files in Pieces

When processing very large files or figuring out the right set of arguments to cor‐ rectly process a large file, you may only want to read in a small piece of a file or iterate through smaller chunks of the file.

(Page 173). 

In [2]:
pd.options.display.max_rows = 10

In [3]:
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [4]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


To read a file in pieces, specify a chunksize as a number of rows:

(Page 174). 

In [38]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

The TextParser object returned by read_csv allows you to iterate over the parts of the file according to the chunksize. For example, we can iterate over ex6.csv, aggre‐ gating the value counts in the 'key' column like so:

(Page 174). 

In [39]:
tot = pd.Series([])
for piece in chunker:
    print(piece['key'].value_counts())
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

S    48
O    44
F    40
J    39
Q    39
     ..
8    13
9    11
2    11
5     9
0     9
Name: key, Length: 36, dtype: int64
O    48
L    44
X    40
I    39
R    38
     ..
1    16
3    16
6    14
2    14
7    12
Name: key, Length: 36, dtype: int64
O    40
A    40
E    39
X    39
M    38
     ..
8    14
4    14
5    14
2    14
1    11
Name: key, Length: 36, dtype: int64
X    43
J    41
V    38
Q    38
D    38
     ..
9    15
0    15
8    13
4    12
7    12
Name: key, Length: 36, dtype: int64
E    54
Q    42
L    40
H    39
P    38
     ..
6    17
8    16
7    13
1     8
9     8
Name: key, Length: 36, dtype: int64
Y    42
F    41
K    41
X    38
V    37
     ..
8    17
3    16
4    13
2    12
0     8
Name: key, Length: 36, dtype: int64
L    41
C    41
W    39
Z    39
X    38
     ..
2    17
1    16
5    15
3    11
6     8
Name: key, Length: 36, dtype: int64
J    42
M    40
X    39
D    39
E    39
     ..
7    14
3    13
5    13
1    12
9     5
Name: key, Length: 36, dtype: int64
O    42


In [40]:
tot = tot.sort_index()

In [41]:
tot

0    151.0
1    146.0
2    152.0
3    162.0
4    171.0
     ...  
V    328.0
W    305.0
X    364.0
Y    314.0
Z    288.0
Length: 36, dtype: float64

In [42]:
tot[:10]

0    151.0
1    146.0
2    152.0
3    162.0
4    171.0
5    157.0
6    166.0
7    164.0
8    162.0
9    150.0
dtype: float64

### Writing Data to Text Format

In [43]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Using DataFrame’s to_csv method, we can write the data out to a comma-separated file:

(Page 175). 

In [45]:
data.to_csv('examples/out.csv')

In [46]:
list(open('examples/out.csv'))

[',something,a,b,c,d,message\n',
 '0,one,1,2,3.0,4,\n',
 '1,two,5,6,,8,world\n',
 '2,three,9,10,11.0,12,foo\n']

Other delimiters can be used, of course (writing to sys.stdout so it prints the text result to the console):

(Page 175). 

In [47]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [49]:
data.to_csv(sys.stdout, sep='|', na_rep='NULL')

|something|a|b|c|d|message
0|one|1|2|3.0|4|NULL
1|two|5|6|NULL|8|world
2|three|9|10|11.0|12|foo


In [50]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [51]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [53]:
dates = pd.date_range('1/1/2000', periods=7)
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')

In [55]:
ts = pd.Series(np.arange(7), index=pd.date_range('1/1/2000', periods=7))
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32

In [56]:
ts.to_csv('examples/tseries.csv')

In [57]:
list(open('examples/tseries.csv'))

['2000-01-01,0\n',
 '2000-01-02,1\n',
 '2000-01-03,2\n',
 '2000-01-04,3\n',
 '2000-01-05,4\n',
 '2000-01-06,5\n',
 '2000-01-07,6\n']

### Working with Delimited Formats

In [58]:
list(open('examples/ex7.csv'))

['"a","b","c"\n', '"1","2","3"\n', '"1","2","3"\n']

It’s possible to load most forms of tabular data from disk using functions like pan das.read_table. In some cases, however, some manual processing may be necessary. It’s not uncommon to receive a file with one or more malformed lines that trip up read_table. To illustrate the basic tools, consider a small CSV file:

(Page 176). 

In [59]:
import csv
f = open('examples/ex7.csv')

reader = csv.reader(f)

In [60]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [61]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

In [62]:
header, values = lines[0], lines[1:]

Then we can create a dictionary of data columns using a dictionary comprehension and the expression zip(*values), which transposes rows to columns:

(Page 177). 

In [63]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

CSV files come in many different flavors. To define a new format with a different delimiter, string quoting convention, or line terminator, we define a simple subclass of csv.Dialect:

(Page 177). 

    class my_dialect(csv.Dialect):
        lineterminator = '\n'
        delimiter = ';'
        quotechar = '"'
        quoting = csv.QUOTE_MINIMAL

    reader = csv.reader(f, dialect=my_dialect)

    reader = csv.reader(f, delimiter='|')

To write delimited files manually, you can use csv.writer. It accepts an open, writa‐ ble file object and the same dialect and format options as csv.reader:

(Page 178). 

    with open('mydata.csv', 'w') as f:
        writer = csv.writer(f, dialect=my_dialect)
        writer.writerow(('one', 'two', 'three'))
        writer.writerow(('1', '2', '3'))
        writer.writerow(('4', '5', '6'))
        writer.writerow(('7', '8', '9'))

### JSON Data

JSON (short for JavaScript Object Notation) has become one of the standard formats for sending data by HTTP request between web browsers and other applications. It is a much more free-form data format than a tabular text form like CSV. Here is an example:

(Page 178). 

In [64]:
obj = """
    {"name": "Wes",
     "places_lived": ["United States", "Spain", "Germany"],
     "pet": null,
     "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
                  {"name": "Katie", "age": 38,
                   "pets": ["Sixes", "Stache", "Cisco"]}]
    }
    """

JSON is very nearly valid Python code with the exception of its null value null and some other nuances (such as disallowing trailing commas at the end of lists). The basic types are objects (dicts), arrays (lists), strings, numbers, booleans, and nulls. All of the keys in an object must be strings. There are several Python libraries for reading
and writing JSON data. I’ll use json here, as it is built into the Python standard library. To convert a JSON string to Python form, use json.loads:


In [65]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 30, 'name': 'Scott', 'pets': ['Zeus', 'Zuko']},
  {'age': 38, 'name': 'Katie', 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [68]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

How you convert a JSON object or list of objects to a DataFrame or some other data structure for analysis will be up to you. Conveniently, you can pass a list of dicts (which were previously JSON objects) to the DataFrame constructor and select a sub‐ set of the data fields:

(Page 179). 

In [67]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


In [69]:
list(open('examples/example.json'))

['[{"a": 1, "b": 2, "c": 3},\n',
 ' {"a": 4, "b": 5, "c": 6},\n',
 ' {"a": 7, "b": 8, "c": 9}]\n']

In [70]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [71]:
print(data.to_json())
print(data.to_json(orient='records'))

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}
[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### XML and HTML: Web Scraping

Python has many libraries for reading and writing data in the ubiquitous HTML and XML formats. Examples include lxml, Beautiful Soup, and html5lib. While lxml is comparatively much faster in general, the other libraries can better handle malformed HTML or XML files. pandas has a built-in function, read_html, which uses libraries like lxml and Beauti‐ ful Soup to automatically parse tables out of HTML files as DataFrame objects. To show how this works, I downloaded an HTML file (used in the pandas documenta‐ tion) from the United States FDIC government agency showing bank failures.1 First, you must install some additional libraries used by read_html:

(Page 180). 

    conda install lxml
    pip install beautifulsoup4 html5lib

The pandas.read_html function has a number of options, but by default it searches for and attempts to parse all tabular data contained within $<table>$ tags. The result is a list of DataFrame objects:

(Page 180). 

In [73]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')
tables

[                             Bank Name             City  ST   CERT  \
 0                          Allied Bank         Mulberry  AR     91   
 1         The Woodbury Banking Company         Woodbury  GA  11297   
 2               First CornerStone Bank  King of Prussia  PA  35312   
 3                   Trust Company Bank          Memphis  TN   9956   
 4           North Milwaukee State Bank        Milwaukee  WI  20364   
 ..                                 ...              ...  ..    ...   
 542                 Superior Bank, FSB         Hinsdale  IL  32646   
 543                Malta National Bank            Malta  OH   6629   
 544    First Alliance Bank & Trust Co.       Manchester  NH  34264   
 545  National State Bank of Metropolis       Metropolis  IL   3815   
 546                   Bank of Honolulu         Honolulu  HI  21029   
 
                    Acquiring Institution        Closing Date  \
 0                           Today's Bank  September 23, 2016   
 1              

In [74]:
len(tables)

1

In [75]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [76]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

#### Parsing XML with lxml.objectify

XML (eXtensible Markup Language) is another common structured data format sup‐ porting hierarchical, nested data with metadata. The book you are currently reading was actually created from a series of large XML documents.

Earlier, I showed the pandas.read_html function, which uses either lxml or Beautiful Soup under the hood to parse data from HTML. XML and HTML are structurally similar, but XML is more general. Here, I will show an example of how to use lxml to parse data from a more general XML format. 

The New York Metropolitan Transportation Authority (MTA) publishes a number of data series about its bus and train services. Here we’ll look at the performance data, which is contained in a set of XML files. Each train or bus service has a different file (like Performance_MNR.xml for the Metro-North Railroad) containing monthly data as a series of XML records that look like this:

(Page 181). 

    <INDICATOR>
      <INDICATOR_SEQ>373889</INDICATOR_SEQ>
      <PARENT_SEQ></PARENT_SEQ>
      <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
      <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
      <DESCRIPTION>Percent of the time that escalators are operational
      systemwide. The availability rate is based on physical observations performed the morning of regular business days only. This is a new indicator the agency began reporting in 2009.</DESCRIPTION>
      <PERIOD_YEAR>2011</PERIOD_YEAR>
      <PERIOD_MONTH>12</PERIOD_MONTH>
      <CATEGORY>Service Indicators</CATEGORY>
      <FREQUENCY>M</FREQUENCY>
      <DESIRED_CHANGE>U</DESIRED_CHANGE>
      <INDICATOR_UNIT>%</INDICATOR_UNIT>
      <DECIMAL_PLACES>1</DECIMAL_PLACES>
      <YTD_TARGET>97.00</YTD_TARGET>
      <YTD_ACTUAL></YTD_ACTUAL>
      <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
      <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
    </INDICATOR>

Using lxml.objectify, we parse the file and get a reference to the root node of the XML file with getroot:

(Page 182). 

In [77]:
from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [78]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [79]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


In [80]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [81]:
root
root.get('href')
root.text

'Google'

## Binary Data Formats

One of the easiest ways to store data (also known as serialization) efficiently in binary format is using Python’s built-in pickle serialization. pandas objects all have a to_pickle method that writes the data to disk in pickle format:

(Page 183). 

In [82]:
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [83]:
frame.to_pickle('examples/frame_pickle')

In [84]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [ ]:
!rm examples/frame_pickle

### Using HDF5 Format

HDF5 is a well-regarded file format intended for storing large quantities of scientific array data. It is available as a C library, and it has interfaces available in many other languages, including Java, Julia, MATLAB, and Python. The “HDF” in HDF5 stands for hierarchical data format. Each HDF5 file can store multiple datasets and support‐ ing metadata. Compared with simpler formats, HDF5 supports on-the-fly compres‐ sion with a variety of compression modes, enabling data with repeated patterns to be stored more efficiently. HDF5 can be a good choice for working with very large data‐ sets that don’t fit into memory, as you can efficiently read and write small sections of much larger arrays. 

While it’s possible to directly access HDF5 files using either the PyTables or h5py libraries, pandas provides a high-level interface that simplifies storing Series and DataFrame object. The HDFStore class works like a dict and handles the low-level details:

(Page 184). 

In [85]:
frame = pd.DataFrame({'a': np.random.randn(100)})
frame

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781
...,...
95,0.795253
96,0.118110
97,-0.748532
98,0.584970


In [91]:
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [92]:
store['obj1']

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781
...,...
95,0.795253
96,0.118110
97,-0.748532
98,0.584970


In [93]:
store['obj1_col']

0    -0.204708
1     0.478943
2    -0.519439
3    -0.555730
4     1.965781
        ...   
95    0.795253
96    0.118110
97   -0.748532
98    0.584970
99    0.152677
Name: a, Length: 100, dtype: float64

HDFStore supports two storage schemas, 'fixed' and 'table'. The latter is generally slower, but it supports query operations using a special syntax:

(Page 185). 

In [94]:
store.put('obj2', frame, format='table')
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,1.007189
11,-1.296221
12,0.274992
13,0.228913
14,1.352917
15,0.886429


> The put is an explicit version of the store['obj2'] = frame method but allows us to set other options like the storage format.



In [95]:
store.close()

In [96]:
frame.to_hdf('mydata.h5', 'obj3', format='table')
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781


In [98]:
import os
os.remove('mydata.h5')

### Reading Microsoft Excel Files

pandas also supports reading tabular data stored in Excel 2003 (and higher) files using either the ExcelFile class or pandas.read_excel function. Internally these tools use the add-on packages xlrd and openpyxl to read XLS and XLSX files, respec‐ tively. You may need to install these manually with pip or conda.

(Page 186). 

In [99]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [100]:
pd.read_excel(xlsx, 'Sheet1')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [101]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [102]:
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [103]:
frame.to_excel('examples/ex2.xlsx')

In [ ]:
!rm examples/ex2.xlsx

## Interacting with Web APIs

Many websites have public APIs providing data feeds via JSON or some other format. There are a number of ways to access these APIs from Python; one easy-to-use method that I recommend is the requests package. 

To find the last 30 GitHub issues for pandas on GitHub, we can make a GET HTTP request using the add-on requests library:

(Page 187). 

In [104]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [105]:
data = resp.json()
data[0]['title']

'to_records() fails for empty dataframes after groupby'

In [106]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

,number,title,labels,state
0,21064,to_records() fails for empty dataframes after ...,[],open
1,21063,segfault in DataFrame.apply with duplicate col...,"[{'id': 697792067, 'url': 'https://api.github....",open
2,21062,Pass sort for agg multiple,[],open
3,21061,DOC: add highlights and toc to whatsnew file f...,"[{'id': 134699, 'url': 'https://api.github.com...",open
4,21060,Add epoch alternative to deprecation message,"[{'id': 134699, 'url': 'https://api.github.com...",open
...,...,...,...,...
25,21023,Feature: footer with aggregate function for pi...,"[{'id': 49254273, 'url': 'https://api.github.c...",open
26,21022,PERF: __contains__ method for Categorical,"[{'id': 78527356, 'url': 'https://api.github.c...",open
27,21020,"Series[categorical] median raises, but DataFra...","[{'id': 35818298, 'url': 'https://api.github.c...",open
28,21019,BUG: CategoricalIndex.searchsorted doesn't ret...,"[{'id': 78527356, 'url': 'https://api.github.c...",open


## Interacting with Databases

In a business setting, most data may not be stored in text or Excel files. SQL-based relational databases (such as SQL Server, PostgreSQL, and MySQL) are in wide use, and many alternative databases have become quite popular. The choice of database is usually dependent on the performance, data integrity, and scalability needs of an application. 

Loading data from SQL into a DataFrame is fairly straightforward, and pandas has some functions to simplify the process. As an example, I’ll create a SQLite database using Python’s built-in sqlite3 driver:

(Page 188). 


In [107]:
import sqlite3

query ="""
    create table test (a varchar(20),
                       b varchar(20),
                       c real,
                       d integer);
"""

In [108]:
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

In [109]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = 'insert into test values(?, ?, ?, ?)'
con.executemany(stmt, data)
con.commit()

In [110]:
cursor = con.execute('select * from test')
rows   = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [111]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

You can pass the list of tuples to the DataFrame constructor, but you also need the column names, contained in the cursor’s description attribute:

(Page 189). 

In [112]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


This is quite a bit of munging that you’d rather not repeat each time you query the database. The SQLAlchemy project is a popular Python SQL toolkit that abstracts away many of the common differences between SQL databases. pandas has a read_sql function that enables you to read data easily from a general SQLAlchemy connection. Here, we’ll connect to the same SQLite database with SQLAlchemy and read data from the table created before:

(Page 190). 

In [113]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [ ]:
!rm mydata.sqlite

## Conclusion